In [1]:
import json
import os
import sys
from litellm import completion
from typing import List, Dict
'''https://console.groq.com/keys'''

'https://console.groq.com/keys'

In [2]:

os.environ['GROQ_API_KEY'] = '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'


In [5]:

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="groq/meta-llama/llama-4-scout-17b-16e-instruct",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content




def extract_markdown_block(response: str, block_type: str = "json") -> str:
    """Extract code block from response"""

    if not '```' in response:
        return response

    code_block = response.split('```')[1].strip()

    if code_block.startswith(block_type):
        code_block = code_block[len(block_type):].strip()

    return code_block






def parse_action(response: str) -> Dict:
    """Parse the LLM response into a structured action dictionary."""
    try:
        response = extract_markdown_block(response, "action")
        response_json = json.loads(response)
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
    except json.JSONDecodeError:
        return {"tool_name": "error", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}




In [6]:
############### DEFINE FUNCTIONS############

def half(value: int):
    halfvalue=value/2
    return halfvalue

def double(value: int):
    doublevalue=value*2
    return doublevalue
    

In [10]:
# Define system instructions (Agent Rules)
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

Available tools:

```json
{
    "half": {
        "description": "Takes an input integer and divides by two.",
        "parameters": {
            "input": {
                "type": "integer"
                "description" : "The integer that is going to be divided by two."
        }
    },
    "double": {
        "description": "Takes an input integer and multiplies by two.",
        "parameters": {
            "file_name": {
                "type": "integer",
                "description": "The integer that is going to be multiplied by two"
            }
        }
    },














    
    "terminate": {
        "description": "Ends the agent loop and provides a summary of the task.",
        "parameters": {
            "message": {
                "type": "string",
                "description": "Summary message to return to the user."
            }
        }
    }
}
```

The user will input an integer and what operation they want to do. If the tool is not available, just terminate.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.

Important!!! Every response MUST have an action.
You must ALWAYS respond in this format:

<Stop and think step by step. Parameters map to args. Insert a rich description of your step by step thoughts here.>

```action
{
    "tool_name": "insert tool_name",
    "args": {...fill in any required arguments here...}
}
```"""
}]

# Initialize agent parameters
iterations = 0
max_iterations = 10

user_task = input("What would you like me to do? ")

memory = [{"role": "user", "content": user_task}]

# The Agent Loop
while iterations < max_iterations:
    # 1. Construct prompt: Combine agent rules with memory
    prompt = agent_rules + memory

    # 2. Generate response from LLM
    print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response: {response}")

    # 3. Parse response to determine action
    action = parse_action(response)
    result = "Action executed"

    if action["tool_name"] == "half":
        result = {"result": half(action["args"]["input"])}
    elif action["tool_name"] == "double":
        result = {"result": double(action["args"]["input"])}




######################################ADD MORE TOOLS




    
    elif action["tool_name"] == "error":
        result = {"error": action["args"]["message"]}
    elif action["tool_name"] == "terminate":
        print(action["args"]["message"])
        break
    else:
        result = {"error": "Unknown action: " + action["tool_name"]}

    print(f"Action result: {result}")

    # 5. Update memory with response and results
    memory.extend([
        {"role": "assistant", "content": response},
        {"role": "user", "content": json.dumps(result)}
    ])

    # 6. Check termination condition
    if action["tool_name"] == "terminate":
        break
    print("ITERATIONNNNNNNNN :", iterations)
    iterations += 1


What would you like me to do?  i need to know what is half of ten


Agent thinking...
Agent response: <Stop and think step by step. The user wants to know what is half of ten. This can be achieved by using the "half" tool. The input for the "half" tool is an integer, which in this case is 10. Parameters map to args, so I need to pass 10 as the input to the "half" tool.>

```action
{
 "tool_name": "half",
 "args": {
 "input": 10
 }
}
```
Action result: {'result': 5.0}
ITERATIONNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN : 0
Agent thinking...
Agent response: <Stop and think step by step. The "half" tool has provided the result, which is 5.0. Now I need to terminate the conversation by using the "terminate" tool and provide a summary message to the user. The summary message will be "Half of 10 is 5.0".>

```action
{
 "tool_name": "terminate",
 "args": {